<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p /content/emoji/
!unzip /content/emojis_am.zip -d /content/emoji/true


In [ ]:
# @title setup data
import os
import random
import torch
import torch.nn as nn
# import torch.nn.parallel
import torchvision
from torchvision import datasets, transforms
import numpy as np

# random.seed(manualSeed)
# torch.manual_seed(manualSeed)

# https://arxiv.org/pdf/1511.06434.pdf
# discriminator strided convolution layers, batch norm_ layers, and LeakyReLU_ activations. 
# input is a 3x64x64 input image and the output is a scalar probability that the input is from the real data distribution. 
# generator convolutional-transpose_ layers, batch norm layers, and ReLU_ activations.


image_size = 64
# transform=transforms.Compose([transforms.Resize(image_size), transforms.CenterCrop(image_size), 
#         transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])
dataset = datasets.ImageFolder(root='/content/emoji', transform=transform)
batch_size = 128
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import matplotlib.pyplot as plt
# real_batch = next(iter(dataloader))
# print(real_batch[0].shape)
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(torchvision.utils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))



In [ ]:
# @title gen disc

nc = 3 # num channels
nz = 100 # Size of z latent vector (i.e. size of generator input)
ngf = 64 # Size of feature maps in generator
ndf = 64 # Size of feature maps in discriminator


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # in_channels, out_channels, kernel_size, stride, padding
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False), nn.BatchNorm2d(ngf * 8), nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False), nn.BatchNorm2d(ngf * 4), nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False), nn.BatchNorm2d(ngf * 2), nn.ReLU(True),
            # nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False), nn.BatchNorm2d(ngf), nn.ReLU(True),
            # nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False), nn.Tanh()

            nn.ConvTranspose2d( ngf * 2, ngf * 2, 4, 2, 1, bias=False), nn.BatchNorm2d(ngf * 2), nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False), nn.BatchNorm2d(ngf), nn.ReLU(True),
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False), nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # in_channels, out_channels, kernel_size, stride, padding
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf * 2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf * 4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf * 8), nn.LeakyReLU(0.2, inplace=True),
            # nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False), nn.Sigmoid()

            nn.Conv2d(ndf * 8, ndf * 8, 4, 2, 1, bias=False), nn.BatchNorm2d(ndf * 8), nn.LeakyReLU(0.2, inplace=True),
            # nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False), nn.BatchNorm2d(ndf * 8), nn.LeakyReLU(0.2, inplace=True),
            # nn.Linear(32, 2), nn.Sigmoid()
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False), nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# custom weights initialization called on netG and netD
# all model weights shall be randomly initialized from a Normal distribution with mean=0, stdev=0.02?
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

netG = Generator().to(device)
# netG = nn.DataParallel(netG, list(range(ngpu)))
netG.apply(weights_init)
# print(netG)

netD = Discriminator().to(device)
netD.apply(weights_init)
# print(netD)
a=0
# strided convolution rather than pooling to downsample lets the network learn its own pooling function.
# batch norm and leaky relu functions promote healthy gradient flow


In [ ]:
# real_cpu=torch.rand(64, 3, 128, 128)
# output = netD(real_cpu).view(-1)
# print(output.shape)

noise = torch.randn(64, nz, 1, 1, device=device)
fake = netG(noise)
print(fake.shape)


torch.Size([64, 3, 128, 128])


In [ ]:
# @title train
def train(dataloader, netG,netD, criterion, optimizerG,optimizerD, scheduler=None, verbose=True):
    for i, data in enumerate(dataloader, 0):
        # train discriminator
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)

        # print("in train: ",output.shape, label.shape)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)

        # print("in train fake.detach(): ",fake.detach().shape)
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

# D(x) - the average output (across the batch) of the discriminator for the all real batch. This should start close to 1 then theoretically converge to 0.5 when G gets better. Think about why this is.
# D(G(z)) - average discriminator outputs for the all fake batch. The first number is before D is updated and the second number is after D is updated. These numbers should start near 0 and converge to 0.5 as G gets better.

        # train generator
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (1 % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(torchvision.utils.make_grid(fake, padding=2, normalize=True))
        # iters += 1




In [ ]:
# @title wwwwwwwwwwwwww
import torch.optim as optim

criterion = nn.MSELoss()
# fixed batch of latent vectors that are drawn from a Gaussian distribution (i.e. fixed_noise)
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

real_label = 1.
fake_label = 0.

# Adam optimizers with learning rate 0.0002 and Beta1 = 0.5.
lr = 0.0002
beta1 = 0.5
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


# Training Loop
# closely follow Algorithm 1 from Goodfellow’s paper, while abiding by some of the best practices shown in ganhacks_.
# https://github.com/soumith/ganhacks

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0



In [ ]:
num_epochs = 5 #200
for epoch in range(num_epochs):
    train(dataloader, netG,netD, criterion, optimizerG,optimizerD)


In [ ]:

plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)
HTML(ani.to_jshtml())


In [ ]:

# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

pthG='/content/drive/MyDrive/ganG.pth'

torch.save(netG.state_dict(), pthG)
# model.load_state_dict(torch.load("model.pth"))



In [ ]:

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# imshow(torchvision.utils.make_grid(images))
images=dataset[0]
print(images)
# imshow(torchvision.utils.make_grid(images))


# real_cpu=torch.rand(64, 3, 128, 128)
# output = netD(real_cpu).view(-1)
# print(output.shape)

noise = torch.randn(64, nz, 1, 1, device=device)
fake = netG(noise)
print(fake.shape)

